In [0]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PySpark Linear Regression") \
    .getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
training = spark.read.format('libsvm').load("/FileStore/tables/sample_linear_regression_data.txt")
training.show()

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
 -9.490009878824548|(10,[0,1,2,3,4,5,...|
 0.2577820163584905|(10,[0,1,2,3,4,5,...|
 -4.438869807456516|(10,[0,1,2,3,4,5,...|
-19.782762789614537|(10,[0,1,2,3,4,5,...|
 -7.966593841555266|(10,[0,1,2,3,4,5,...|
 -7.896274316726144|(10,[0,1,2,3,4,5,...|
 -8.464803554195287|(10,[0,1,2,3,4,5,...|
 2.1214592666251364|(10,[0,1,2,3,4,5,...|
 1.0720117616524107|(10,[0,1,2,3,4,5,...|
-13.772441561702871|(10,[0,1,2,3,4,5,...|
 -5.082010756207233|(10,[0,1,2,3,4,5,...|
 7.887786536531237|(10,[0,1,2,3,4,5,...|
 14.323146365332388|(10,[0,1,2,3,4,5,...|
-20.057482615789212|(10,[0,1,2,3,4,5,...|
-0.8995693247765151|(10,[0,1,2,3,4,5,...|
 -19.16829262296376|(10,[0,1,2,3,4,5,...|
 5.601801561245534|(10,[0,1,2,3,4,5,...|
-3.2256352187273354|(10,[0,1,2,3,4,5,...|
 1.5299675726687754|(10,[0,1,2,3,4,5,...|
 -0.250102447941961|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

In [0]:
lr = LinearRegression(featuresCol = 'features', labelCol = 'label', predictionCol = 'prediction' ) # put whatevert real name

In [0]:
ldModel = lr.fit(training)

In [0]:
ldModel.coefficients

Out[6]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [0]:
ldModel.intercept

Out[8]: 0.14228558260358093

In [0]:
#my_data = my_data.drop(*['Batsman', 'Bowler', 'Id'])
## import sql function pyspark
# import pyspark.sql.functions as f

# # null values in each column
# data_agg = my_data.agg(*[f.count(f.when(f.isnull(c), c)).alias(c) for c in my_data.columns])
# data_agg.show()

Out[13]: <pyspark.ml.regression.LinearRegressionTrainingSummary at 0x7fc6dc0c7ed0>

In [0]:
training_summary = ldModel.summary

In [0]:
training_summary.rootMeanSquaredError

Out[14]: 10.16309157133015

In [0]:
training_summary.r2

Out[15]: 0.027839179518600154

In [0]:
# this is copied from https://runawayhorse001.github.io/LearningApacheSpark/regression.html
# thank George Feng for such a good spark learning file.

def modelsummary(model):
    import numpy as np
    print ("Note: the last rows are the information for Intercept")
    print ("##","-------------------------------------------------")
    print ("##","  Estimate   |   Std.Error | t Values  |  P-value")
    coef = np.append(list(model.coefficients),model.intercept)
    Summary=model.summary

    for i in range(len(Summary.pValues)):
        print ("##",'{:10.6f}'.format(coef[i]),\
        '{:10.6f}'.format(Summary.coefficientStandardErrors[i]),\
        '{:8.3f}'.format(Summary.tValues[i]),\
        '{:10.6f}'.format(Summary.pValues[i]))

    print ("##",'---')
    print ("##","Mean squared error: % .6f" \
           % Summary.meanSquaredError, ", RMSE: % .6f" \
           % Summary.rootMeanSquaredError )
    print ("##","Multiple R-squared: %f" % Summary.r2, ", \
            Total iterations: %i"% Summary.totalIterations)

In [0]:
modelsummary(ldModel)

Note: the last rows are the information for Intercept
## -------------------------------------------------
## Estimate | Std.Error | t Values | P-value
## 0.007335 0.806880 0.009 0.992751
## 0.831376 0.817505 1.017 0.309671
## -0.809531 0.809569 -1.000 0.317827
## 2.441192 0.843518 2.894 0.003972
## 0.519171 0.806692 0.644 0.520149
## 1.153459 0.823868 1.400 0.162130
## -0.298912 0.804191 -0.372 0.710282
## -0.512851 0.809510 -0.634 0.526681
## -0.619713 0.842900 -0.735 0.462560
## 0.695615 0.788760 0.882 0.378258
## 0.142286 0.464058 0.307 0.759269
## ---
## Mean squared error: 103.288430 , RMSE: 10.163092
## Multiple R-squared: 0.027839 , Total iterations: 1

In [0]:
all_data = spark.read.format('libsvm').load("/FileStore/tables/sample_linear_regression_data.txt")

In [0]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [0]:
train_data.describe().show()

+-------+-------------------+
summary| label|
+-------+-------------------+
 count| 365|
 mean| 0.6847431262776856|
 stddev| 10.393530584761306|
 min|-26.805483428483072|
 max| 27.78383192005107|
+-------+-------------------+

In [0]:
test_data.describe().show()

+-------+-------------------+
summary| label|
+-------+-------------------+
 count| 136|
 mean|-0.8913966262622494|
 stddev| 10.05980292176248|
 min|-28.571478869743427|
 max| 24.290551295953957|
+-------+-------------------+

In [0]:
correct_model = lr.fit(train_data)

In [0]:
test_results = correct_model.evaluate(test_data)

In [0]:
test_results.r2

Out[37]: -0.0698111497067948

In [0]:
test_results.rootMeanSquaredError

Out[38]: 10.366698375026026

In [0]:
unlabeled_data = test_data.select('features')

In [0]:
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows

In [0]:
predictions = correct_model.transform(unlabeled_data)

In [0]:
predictions.show()  # when deplot to predict new data, we can't evaluate yet we don't have true data to evaluate against

+--------------------+--------------------+
 features| prediction|
+--------------------+--------------------+
(10,[0,1,2,3,4,5,...| -0.5640515061208576|
(10,[0,1,2,3,4,5,...| -0.8044983227026771|
(10,[0,1,2,3,4,5,...| -2.8676499655069545|
(10,[0,1,2,3,4,5,...| 1.4243059189578666|
(10,[0,1,2,3,4,5,...| 1.7170746963586487|
(10,[0,1,2,3,4,5,...| 1.4614638911741058|
(10,[0,1,2,3,4,5,...| -1.2179047864021122|
(10,[0,1,2,3,4,5,...| 3.285960132822241|
(10,[0,1,2,3,4,5,...|-0.09722170481827319|
(10,[0,1,2,3,4,5,...| 3.1327092172740834|
(10,[0,1,2,3,4,5,...| 1.9757159365244021|
(10,[0,1,2,3,4,5,...| 1.0602376142336614|
(10,[0,1,2,3,4,5,...| 0.15463823325722825|
(10,[0,1,2,3,4,5,...| -3.4007461214545214|
(10,[0,1,2,3,4,5,...| 1.8918871742704053|
(10,[0,1,2,3,4,5,...| 2.0630047115331918|
(10,[0,1,2,3,4,5,...| 4.939006837317798|
(10,[0,1,2,3,4,5,...| 2.317195567156284|
(10,[0,1,2,3,4,5,...| 1.3401189187153777|
(10,[0,1,2,3,4,5,...| -2.5659869691698796|
+--------------------+--------------------+
only showing top 20 rows

In [0]:
 from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [0]:
spark = SparkSession \
    .builder \
    .appName("PySpark Linear Regression") \
    .getOrCreate()


In [0]:
data = spark.read.csv("/FileStore/tables/Ecommerce_Customers.csv", inferSchema = True, header = True)

In [0]:
data.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [0]:
data.head(1)[0]

Out[8]: Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [0]:
for item in data.head(1)[0]:
  print (item)  # the real value

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
# care numerical feature for now
data.columns

Out[11]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [0]:
assembler = VectorAssembler(inputCols = [
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'], 
  outputCol = 'features')

In [0]:
output = assembler.transform(data)

In [0]:
output.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
output.head(1)

Out[20]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [0]:
final_data = output.select('features','Yearly Amount Spent')

In [0]:
final_data.show()

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[34.4972677251122...| 587.9510539684005|
[31.9262720263601...| 392.2049334443264|
[33.0009147556426...| 487.54750486747207|
[34.3055566297555...| 581.8523440352177|
[33.3306725236463...| 599.4060920457634|
[33.8710378793419...| 637.102447915074|
[32.0215955013870...| 521.5721747578274|
[32.7391429383803...| 549.9041461052942|
[33.9877728956856...| 570.2004089636196|
[31.9365486184489...| 427.1993848953282|
[33.9925727749537...| 492.6060127179966|
[33.8793608248049...| 522.3374046069357|
[29.5324289670579...| 408.6403510726275|
[33.1903340437226...| 573.4158673313865|
[32.3879758531538...| 470.4527333009554|
[30.7377203726281...| 461.7807421962299|
[32.1253868972878...| 457.84769594494855|
[32.3388993230671...| 407.70454754954415|
[32.1878120459321...| 452.3156754800354|
[32.6178560628234...| 605.061038804892|
+--------------------+-------------------+
only showing top 20 rows

In [0]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [0]:
train_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 348|
 mean| 501.5693212120168|
 stddev| 78.91193352979212|
 min| 282.4712457199145|
 max| 725.5848140556806|
+-------+-------------------+

In [0]:
test_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 152|
 mean| 494.15062728627333|
 stddev| 80.25219422056426|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [0]:
lr = LinearRegression(labelCol = 'Yearly Amount Spent')  # use default values if not specified

In [0]:
lr_model = lr.fit(train_data)

In [0]:
test_results = lr_model.evaluate(test_data)

In [0]:
test_results.residuals.show()

+-------------------+
 residuals|
+-------------------+
 11.220451109656096|
 11.388743350607797|
-15.366986051026402|
-3.5130360112213452|
 -7.269486609214937|
 11.306488600974035|
 19.424884083787276|
 4.378974764966301|
 -5.064769046465017|
 -2.981101435641051|
-17.493079062400284|
 7.28553844462175|
-26.094105951318284|
 -7.103589524102858|
 2.5211419404702724|
 -4.32961885088622|
 8.075253086727287|
 -8.648243515564388|
 -1.583900198166134|
 -16.80462546084533|
+-------------------+
only showing top 20 rows

In [0]:
test_results.rootMeanSquaredError

Out[33]: 10.71072901949648

In [0]:
test_results.r2

Out[34]: 0.9820695630427302

In [0]:
final_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 500|
 mean| 499.3140382585909|
 stddev| 79.3147815497068|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [0]:
unlabeled_data = test_data.select('features')

In [0]:
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
[29.5324289670579...|
[30.7377203726281...|
[30.8162006488763...|
[30.8364326747734...|
[31.1280900496166...|
[31.1695067987115...|
[31.3123495994443...|
[31.3662121671876...|
[31.5147378578019...|
[31.5171218025062...|
[31.5702008293202...|
[31.6548096756927...|
[31.6739155032749...|
[31.7207699002873...|
[31.7366356860502...|
[31.7656188210424...|
[31.8209982016720...|
[31.8279790554652...|
[31.8627411090001...|
[31.9048571310136...|
+--------------------+
only showing top 20 rows

In [0]:
predictions = lr_model.transform(unlabeled_data)
predictions .show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[29.5324289670579...| 397.4198999629714|
[30.7377203726281...| 450.3919988456221|
[30.8162006488763...| 281.4533269994954|
[30.8364326747734...|471.01493643821095|
[31.1280900496166...| 564.5221733562696|
[31.1695067987115...| 416.0500422013188|
[31.3123495994443...|444.16653394415334|
[31.3662121671876...| 426.2099077915186|
[31.5147378578019...|494.87725704292643|
[31.5171218025062...|278.89952208602676|
[31.5702008293202...| 563.4385712038052|
[31.6548096756927...|467.97788528292676|
[31.6739155032749...| 501.8191738611995|
[31.7207699002873...| 545.8785230021258|
[31.7366356860502...|494.41230431506165|
[31.7656188210424...|500.88370048649335|
[31.8209982016720...|416.60002792648606|
[31.8279790554652...| 448.6509910625059|
[31.8627411090001...| 557.8820413722128|
[31.9048571310136...|490.75448288366147|
+--------------------+------------------+
only showing top 20 rows

In [0]:
 data = spark.read.csv("/FileStore/tables/cruise_ship_info.csv", header = True, inferSchema = True)

In [0]:
data.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [0]:
for ship in data.head(5):
  print(ship)
  print("\n")

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)

In [0]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
 Cruise_line|count|
+-----------------+-----+
 Costa| 11|
 P&O| 6|
 Cunard| 3|
Regent_Seven_Seas| 5|
 MSC| 8|
 Carnival| 22|
 Crystal| 2|
 Orient| 1|
 Princess| 17|
 Silversea| 4|
 Seabourn| 3|
 Holland_American| 14|
 Windstar| 3|
 Disney| 2|
 Norwegian| 13|
 Oceania| 3|
 Azamara| 2|
 Celebrity| 10|
 Star| 6|
 Royal_Caribbean| 23|
+-----------------+-----+

In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
indexer = StringIndexer(inputCol = 'Cruise_line', outputCol = 'cruise_cat')
indexed = indexer.fit(data).transform(data)

In [0]:
indexed.head(1)  # label encoding

Out[49]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)]

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
indexed.columns

Out[51]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [0]:
assembler = VectorAssembler(inputCols= ['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density', 
 'cruise_cat'], 
  outputCol = 'features'
)

In [0]:
output = assembler.transform(indexed)

In [0]:
output.select('features', 'crew').show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [0]:
final_data =output.select('features', 'crew')

In [0]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [0]:
 train_data.describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 113|
 mean|7.490796460176999|
 stddev|3.475199414009246|
 min| 0.59|
 max| 19.1|
+-------+-----------------+

In [0]:
 test_data.describe().show()

+-------+------------------+
summary| crew|
+-------+------------------+
 count| 45|
 mean| 8.556|
 stddev|3.4965602577910238|
 min| 0.88|
 max| 21.0|
+-------+------------------+

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
lr = LinearRegression(labelCol = 'crew')

In [0]:
lr_model = lr.fit(train_data)

In [0]:
train_model = lr_model.evaluate(test_data)

In [0]:
train_model.r2

Out[66]: 0.94162867981505

In [0]:
train_model.rootMeanSquaredError  # this is pretty book looking below

Out[68]: 0.8353353421273769

In [0]:
train_data.describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 113|
 mean|7.490796460176999|
 stddev|3.475199414009246|
 min| 0.59|
 max| 19.1|
+-------+-----------------+

In [0]:
train_model.meanAbsoluteError

Out[70]: 0.6242605948207921

In [0]:
from pyspark.sql.functions import corr #pearson correlation
data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
summary|Ship_name|Cruise_line| Age| Tonnage| passengers| length| cabins|passenger_density| crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
 count| 158| 158| 158| 158| 158| 158| 158| 158| 158|
 mean| Infinity| null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
 stddev| NaN| null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
 min|Adventure| Azamara| 4| 2.329| 0.66| 2.79| 0.33| 17.7| 0.59|
 max|Zuiderdam| Windstar| 48| 220.0| 54.0| 11.82| 27.0| 71.43| 21.0|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+

In [0]:
data.select(corr('crew', 'passengers')).show()

+----------------------+
corr(crew, passengers)|
+----------------------+
 0.9152341306065384|
+----------------------+

In [0]:
data.select(corr('crew', 'cabins')).show()

+------------------+
corr(crew, cabins)|
+------------------+
0.9508226063578497|
+------------------+